In [82]:
import pdftables_api
import pandas as pd 
import os
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import nltk
import re
import pymongo
from pymongo import MongoClient
import pprint
import mysql.connector
from mongoengine import connect
from mongoengine import Document, ListField, StringField, URLField


In [83]:
# provide the path to the html files
all_CVs_path = os.listdir('/home/yaba/Documents/projetBDD/CVs_converted/format_HTML')  


In [84]:
def call_api():
    api_key ='zptbl1cmpnf3'
    # convert all CVs from pdf to other formats like xml/csv/html
    cv_parser = pdftables_api.Client(api_key)
    for cv_name in all_CVs:
         if(cv_name[-4:]=='.pdf'):
            #cv_parser.csv(cv_name,cv_name+'_convert')
            #cv_parser.xml(cv_name,cv_name+'_convert')
            cv_parser.html(cv_name,cv_name+'_convert')

In [86]:
def extract_resume_info(resumes_path):
    # get the names of all resumes files
    all_CVs = os.listdir(resumes_path) 
    resumes_list=[]
    # loop through resumes using the file name
    for cv_name in all_CVs:
       puzzle_pieces=[]
       #open the resume 
        with open(resumes_path+'/'+cv_name, "r", encoding='utf-8') as f:
            text= f.read()
       # parse the html-encoded resume using BeautifulSoup 
       html_page = BeautifulSoup(text,"html.parser")
       # extract the content inside each TD tage
       for td in html_page.find_all(name="td"):
            info_piece=td.text
            puzzle_pieces.append(info_piece)
       resumes_list.append(puzzle_pieces)

    return resumes_list
    

resumes_content = (extract_resume_info('/home/yaba/Documents/projetBDD/CVs_converted/format_HTML'))
    

In [87]:
from os import walk
import itertools
# extract the names of the candidates 
def extract_candidate_names(resumes_path):
    all_CVs = os.listdir(resumes_path) 
    first_last_names=[]
    final_output=[]
    for (dirpath, dirnames, filenames) in walk(resumes_path):
        candidates_names = filenames
    
    for candidate_name in candidates_names :
        split_1 = candidate_name.split('.')
        first_last_names.append(split_1[0].split('CV')[1].replace('_',' ').replace('-',' ')[1:])

    for last_first_name in first_last_names :
        #name_pieces = last_first_name.split(' ')
        last_name = [' '.join(b) for a, b in itertools.groupby(last_first_name.split(), key=str.isupper) if a]
        first_name=  last_first_name.replace(last_name[0],'')
        last_name.append(first_name)
        final_output.append(last_name)
        
    
    #print(final_output[0][1])
    return final_output
        
        
extract_candidate_names('C:/Users/HAITAM/Desktop/CVs_converted/format_HTML')


[['MBENGUE', ' Ibrahima'],
 ['ADEL', ' SalahEddine'],
 ['ALSADIK', ' Ammar'],
 ['AOUDJEHANE', ' Sarah'],
 ['BABORI', ' Yasmine'],
 ['BENSAID', ' Imane'],
 ['BERHILI', ' Faïza'],
 ['BIAN', ' Yiping'],
 ['BOUHOUT', ' Adil'],
 ['BOUMAZA', ' Heytem'],
 ['BOUSSAC', ' Emiliano'],
 ['CHAMBRIER', ' Julian'],
 ['CHAOUCHE', ' Yacine'],
 ['CHATTAT', ' Naima'],
 ['CHFADI', ' Sara'],
 ['DJENADI', ' Sabrina'],
 ['EL HAIL', ' Mohamed'],
 ['ESSEBBABI', ' Nour'],
 ['GUASSIM', ' Mohamed'],
 ['HAIDOURI', ' Issam'],
 ['HJIB', ' Maroua'],
 ['JIANG', ' Alexis'],
 ['KARAKAS', ' Alexandre'],
 ['KOUACHI', ' Abdeldjalil'],
 ['LE', ' Minh Hao'],
 ['MALKI', ' Salma'],
 ['MEDJDOUBI', ' Lynda'],
 ['NDIAYE', ' Mohamed'],
 ['OULDHADDA', ' Tiziri'],
 ['OULDOULHADJ', ' Dehbia'],
 ['QIAN', ' Xiaotong'],
 ['SABOUNI', ' Soukayna'],
 ['SATHIANATHAN', ' Sayanthan'],
 ['SENE', ' Aïssatou Ba'],
 ['TCHEDRE', ' Napo Tiyadja'],
 ['TERAA', ' Youcef Oussama'],
 ['YABA', ' Michel'],
 ['SENE', ' Mame astou']]

In [88]:

# keywords for detecting in-section heading
address_keywords=["RUE","BOULEVARD","AVENUE","QUAI","IMPASSE","PONT","SQUARE","ALLÉE","ALLE","ALLÉES","VOIE","MONTÉE","ESPLANADE","ROUTE","VOIRIE","CITÉ","CHEMIN","PARVIS"]
#----------utility methods ---------
#removing empty string elements from resume lists 
def remove_emp_u_strings(resumes_list):
    new_cleaned_resume_list = [remove_u_strings(resume) for resume in resumes_list]
    new_cleaned_resume_list = [list(filter(None,resume)) for resume in resumes_list]
    return  new_cleaned_resume_list


#this function takes a list of strings and remove all occurences of special characters within that string
def remove_u_strings(resume_content):
  
    for indice, content in enumerate(resume_content):
        resume_content[indice] = (resume_content[indice].encode("ascii","ignore")).decode()
        #extracting the \u~ words from every single element in the resume content list
    return resume_content


# this method uses regular expressions to extract infos from the resume.
# it loops through each resume exactly once.
def extract_contact_info(resume_content):

    contact_info = {
        'email':[],
        'phone':[],
        'address':[],
        'linkedin':[],
        'github':[],
        'twitter':[],
        'facebook':[],
        'social_status':[],
        'driving_licence':[],
        'nationality':[],
        'website':[],
        'age':[]
    }

    for line in resume_content:
        #email
        match_e = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', line)
        if match_e:
           contact_info['email'].append(''.join(match_e))
        #phone number
        match_n = re.findall(r'^(?:(?:\+|00)33|0)s*[1-9](?:[\s.-]*\d{2}){4}$', line)
        if match_n:
           contact_info['phone'].append(' '.join(match_n))
        #address
        if any(contains_this_word(line.upper(),adr_keyword) for adr_keyword in address_keywords) \
        or len(re.findall(r'^[0-9]{5}(?:-[0-9]{4})?$',line.upper())) > 0: 
           contact_info['address'].append(line)

        #linkedin
        found,w=word_in_line(line,'LINKEDIN.COM')
        if(found):
            contact_info['linkedin'].append(w)
        
        #github   
        found,w = word_in_line(line,'GITHUB.COM')
        if(found):
            contact_info['github'].append(w)
        
        #twitter
        
        #facebook

        #social status / marié (M), pacsé (P), divorcé (D), séparé (D), célibataire (C) ou veuf (V).
        social_status = ['marié','pacsé','divorcé','séparé','célibataire','veuf']
        if any(contains_this_word(line.upper(),s_s_keyword.upper()) for s_s_keyword in social_status):
            contact_info['social_status'].append(s_s_keyword[0].upper())

        #driving licence
        if contains_this_word(line,'permis'.upper()):
            contact_info['driving_licence']    

        #website 
        match_w = re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', line)
        if match_w :
            
            contact_info['website'].append(''.join(list(match_w.group())))
        
        #age 
        match_a = re.search(r'\d{2} *ANS',line.upper())
        if match_a:
           contact_info['age'].append(''.join(list(match_a.group())))
        
        
    
    return  contact_info

In [89]:
#keywords dictonnary (used to extract infos), can be easily extended and improved with new keywords
keywords_dic = {
   
   'introduction': {
       'keywords_fr': ['RECHERCHE','CANDIDATURE','CANDIDAT','INTRODUCTION'],
        'keywords_en': ['CANDIDATE','CANDIDANCY']
   },
   'formation':{
     'keywords_fr': ['FORMATION','DIPLOME','MASTER','DOCTORAT','LICENCE'],
        'keywords_en': ['TRAINING','MASTERS','DOCTORATE','INTERNSHIP','BACHELOR']
   },
    'etablissement':{
     'keywords_fr': ['UNIVERSITE','INTITUT','ECOLE','LYCEE'],
        'keywords_en': ['UNIVERSITY','SCHOOL','INSTITUTE']
   },
   'experience':{
        'keywords_fr': ['EXPERIENCE','EMPLOI','EXPERIENCES PROFESSIONNELLES','STAGE'],
        'keywords_en': ['EXPERIENCE']

   },
   'interet':{
        'keywords_fr': ['LOISIRS','INTERET','CENTRE D''INTERET','CENTRE'],
        'keywords_en': ['HOBBIES','INTEREST']
   },
   'projets':{
        'keywords_fr': ['PROJETS','REALISATION','PROJET ACADEMIQUES'],
        'keywords_en': ['PROJECT','ACADEMIC PROJECTS']
   },
   'competences':{
        'keywords_fr': ['COMPETENCE','OUTILS'],
        'keywords_en': ['SKILLS','TOOLS']
   },
   'langues':{
        'keywords_fr': ['LANGUE','COMPETENCES LINGUISTIQUES','LU', 'ECRIT', 'PARLE','DEBUTANT','FRANCAIS','ANGLAIS','ARABE','CHINOIS','BILINGUE','NATALE','MATERNELLE','COURANT'],
        'keywords_en': ['LINGUISTIC SKILLS']
   },
   'certificats':{
        'keywords_fr': ['CERTIFICAT','CERTIFICATION'],
        'keywords_en': ['CERTIFICATE']

   },
   'references':{
        'keywords_fr': ['REFERENCES'],
        'keywords_en': ['REFERENCES']
   },
   'atouts':{
        'keywords_fr': ['ATOUT'],
        'keywords_en': []
   },
   'parascolaires':{
        'keywords_fr': ['PARASCOLAIRE','PARASCOLAIRES'],
        'keywords_en': []
   }
}





In [90]:

def contains_this_word(s, w):
    return f' {w} ' in f' {s} '

def word_in_line(s,w):
    word_list = s.split(' ')
    for word in word_list:
        if(f' {w} ' in f' {word}'):
            return True,w
    
    return False,''

#this function takes a string as an argument and returns a list of words formed from this string minus every character it  #contains
def minus_one_letter(word):
    word_parts =[word]
    for index,v in enumerate(word):
        word_parts.append(''.join([word[i] for i in range(len(word)) if i != index]))
    return word_parts
    

#this function takes a line from a resume and returns the section to which it belongs if it exists otherwise
#it returns False
def find_which_section(line):
    section = ''
    indicator= False
    for section_name in keywords_dic:
        for keyword_lan in keywords_dic[section_name]:
           # for element in keywords_dic[section_name][keyword_lan]:
            for element in keywords_dic[section_name][keyword_lan]:
               #get all possible combinations of each keyword
                list_elements = minus_one_letter(element)
                if any(contains_this_word(line.upper(),element_part) for element_part in list_elements):
                    section = section_name
                    indicator= True
                    return section,indicator
    return section , indicator


#extract maximum amount of content from each section in the resmue
def extract_infos(resumes_content):
    candidates = []
   
    for resume in resumes_content:
        content_dic = {
         'introduction': [],
         'contact':{},
         'formation':[],
         'etablissement':[],
         'experience': [],
         'interet':[],
         'projets':[],
         'competences': [],
         'langues':[],
         'certificats': [],  
         'references':[],
         'atouts':[],
         'parascolaires':[],
         'other':[]
         }
        previous_section = 'other'
        for line in resume:
            # check if this line is the start of a particular section in the resume
            section,indicator = find_which_section(line)
            if(indicator):
                # a new section has been found
                previous_section=section
                content_dic[section].append(''.join(line))
            else:
                content_dic[previous_section].append(''.join(line))
        # extract contact infos
        content_dic['contact'] = extract_contact_info(resume)
        # append the content dictionary to the list of candidates
        
        candidates.append(content_dic)
        
    return candidates  

In [91]:


cleaned_resumes_content= remove_emp_u_strings(resumes_content)

#ages = [extract_contact_info(cv) for cv in cleaned_resumes_content]
candidates_list = extract_infos(cleaned_resumes_content)



In [92]:

print("====================================")
print("=====================================")
print(candidates_list)


[{'introduction': ['En recherche de stage de fin dtude dune', 'AIS TECH / JUIL 2020  SEPT 2020', 'dure de 6 mois en tant que cloud', 'data', 'DAKAR, SENEGAL', 'engineer,  partir du mois davril 2021', 'INGENIEUR CLOUD DEVOPS', 'Dploiement et optimisation des supports client du rseau', 'de transfert', 'Money', 'sur', 'un', 'environnement', 'cloud', 'on-', 'LANGUES', 'premise.'], 'contact': {'email': ['Ibrahima.mbengue1@edu.univ-paris13.fr'], 'phone': ['06 27 51 72 87'], 'address': ['2, alle des tilleuls'], 'linkedin': [], 'github': [], 'twitter': [], 'facebook': [], 'social_status': [], 'driving_licence': [], 'nationality': [], 'website': [], 'age': []}, 'formation': ['MASTER 2, EXPLORATION INFORMATIQUE DE', '2, alle des tilleuls', 'DONNES ET DCISIONNEL', '93330, Neuilly Sur Marne', 'FORMATION', 'CERTIFICATIONS', 'M2 INFORMATIQUE DECISIONNEL/ DEPUIS SEPT 2020', 'PROFESSIONAL CLOUD DATA ENGINEER WITH GCP', 'Master 2 en exploration informatique de donnes et', 'dcisionnel  USPN de Paris', '

In [93]:
def insert_mongodb(hostname,port_number,candidates_list):
    # connecting to DB
    client = MongoClient(host=hostname, port=port_number)
       
    #add the names to the candidates 
    names_list = extract_candidate_names('/home/yaba/Documents/projetBDD/CVs_converted/format_HTML')
       
       
    name_counter = 0 
    #add the names to the extracted infos
    for candidate in candidates_list:
        candidate['prenom']= names_list[name_counter][1]
        candidate['nome']= names_list[name_counter][0]
        name_counter = name_counter + 1

    cv_db = client.cv_etudiants
    CVs_collection=cv_db.CVs

    # cancatenate the items of each list
    CVs_collection.insert_many(candidates_list)

    # print the elements that were inserted 
    for cv in CVs_collection.find():
        print(cv)

    client.close()

In [81]:
insert_mongodb('localhost',27017,candidates_list)

 Ibrahima
{'_id': ObjectId('6053524f5cd836894ce285b5'), 'introduction': ['En recherche de stage de fin dtude dune', 'AIS TECH / JUIL 2020  SEPT 2020', 'dure de 6 mois en tant que cloud', 'data', 'DAKAR, SENEGAL', 'engineer,  partir du mois davril 2021', 'INGENIEUR CLOUD DEVOPS', 'Dploiement et optimisation des supports client du rseau', 'de transfert', 'Money', 'sur', 'un', 'environnement', 'cloud', 'on-', 'LANGUES', 'premise.'], 'contact': {'email': ['Ibrahima.mbengue1@edu.univ-paris13.fr'], 'phone': ['06 27 51 72 87'], 'address': ['2, alle des tilleuls'], 'linkedin': [], 'github': [], 'twitter': [], 'facebook': [], 'social_status': [], 'driving_licence': [], 'nationality': [], 'website': [], 'age': []}, 'formation': ['MASTER 2, EXPLORATION INFORMATIQUE DE', '2, alle des tilleuls', 'DONNES ET DCISIONNEL', '93330, Neuilly Sur Marne', 'FORMATION', 'CERTIFICATIONS', 'M2 INFORMATIQUE DECISIONNEL/ DEPUIS SEPT 2020', 'PROFESSIONAL CLOUD DATA ENGINEER WITH GCP', 'Master 2 en exploration info

In [105]:
# te isnert method that takes as arguments the CVs list and connection credentials 
def insert_into_mysql_db(cvs_list,hostname,username,passw):
    
    #opening the connection to the database
    mydb = mysql.connector.connect(
    host=hostname,
    user=username,
    password=passw
    )
    global_id_generator = 100
    my_cursor = mydb.cursor()
    
    # get the first and last names
    first_last_names = extract_candidate_names('/home/yaba/Documents/projetBDD/CVs_converted/format_HTML')
    name_counter = 0

    for cv in cvs_list :
    # table candidat
        sql_select_database ="use candidats"
  
        # queries
        sql_candidat = "INSERT INTO `candidat` (`ID_CANDIDAT`, `NOM_CANDIDAT`, `PRENOM_CANDIDAT`, `AGE_CANDIDAT`,`SEXE`, `DATE_NAISSANCE`, `SITUATION_FAMILIALE`, `PERMIS`, `NATIONALITE`, `MOBILITE`, `PRESENTATION`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        sql_etablissement = "INSERT INTO `etablissement` (`ID_ETABLISSEMENT`, `NOM_ETABLISSEMENT`, `TYPE_ETABLISSEMENT`,  `VILLE_ETABLISSEMENT`, `PAYS_ETABLISSEMENT`) VALUES (%s,%s,%s,%s,%s)"
        sql_formation = 'INSERT INTO FORMATIONS VALUES (%s,%s,%s)'
        sql_suivre_formation = 'INSERT INTO SUIVRE_FORMATION VALUES (%s,%s,%s,%s,%s,%s)'
        sql_langues = 'INSERT INTO LANGUES VALUES (%s,%s,%s)'
        sql_parler_langues = 'INSERT INTO PARLE_LANGUES VALUES (%s,%s,%s,%s)'
        sql_centre_interet = 'INSERT INTO CENTRE_INTERET VALUES (%s,%s)'
        sql_aimer_interet = 'INSERT INTO AIMER_INTERET VALUES (%s,%s,%s)'
        sql_certificat = 'INSERT INTO CERTIFICAT VALUES (%s,%s,%s,%s,%s,%s,%s)'
        sql_competence_operationnelles = 'INSERT INTO COMPETENCES_OPERATIONNELLES VALUES (%s,%s,%s)'
        sql_posseder_comp_op = 'INSERT INTO POSSEDER_COMP_OP VALUES (%s,%s,%s)'
        sql_competence_technique = 'INSERT INTO COMPETENCES_TECHNIQUES VALUES (%s,%s,%s)'
        sql_posseder_comp_te = 'INSERT INTO POSSEDER_COMP_TECH VALUES (%s,%s,%s,%s)'
        sql_relation = 'INSERT INTO RELATION VALUES (%s,%s)'
        sql_reference = 'INSERT INTO REFERENCE VALUES (%s,%s,%s,%s,%s)'
        sql_contacts = 'INSERT INTO CONTACTS VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        sql_experience_professionnelle = 'INSERT INTO EXPERIENCE_PROFESSIONNELLE VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        sql_projets = 'INSERT INTO PROJETS VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

        # query values
        #candidat
        current_id_candidat = global_id_generator
        other_dic = ''.join(cv['other'])
        intro = ''.join(cv['introduction'])
        contact_dic = cv['contact']
        age_string = contact_dic['age']
        print("inserted",name_counter)

        age_number = re.findall(r'\d+', ''.join(age_string))
        if (not age_number):
            age_number=None
        else:
            age_number=int(age_number[0])
            
        intermediate_y = first_last_names[name_counter]
        first_name =intermediate_y[0]
        last_name =intermediate_y[1]
        name_counter += 1
        val_candidat = (current_id_candidat,first_name,last_name,age_number,None,None,None,None,None,None,intro)
        
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_candidat, val_candidat)
            mydb.commit()
            global_id_generator += 1
        except Error as e :
            print('Error : ',e)
    
        #contact 
        id_contact = global_id_generator
        contact_dic = cv['contact']
        val_contacts = (id_contact,None,''.join(contact_dic['address']),None,None,None,' '.join(contact_dic['phone']),' '.join(contact_dic['email']),None,''.join(contact_dic['linkedin']),None,None,None,current_id_candidat)
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_contacts, val_contacts)
            mydb.commit()
            global_id_generator += 1
  
         except Error as e :
           print('Error : ',e)

        
       #etablissement
        current_id_etablissement = global_id_generator
        val_etablissement = (current_id_etablissement,' '.join(cv['etablissement']),None,None,None)
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_etablissement, val_etablissement)
            mydb.commit()
            global_id_generator += 1
        except Error as e :
            print('Error : ',e)

        #formation
        current_code_formation = ' '+str(global_id_generator) # actually it must be extracted and compared with previous codes
        val_formation =(current_code_formation,current_id_etablissement,' '.join(cv['formation']))
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_formation, val_formation)
            mydb.commit()
        except Error as e :
            print('Error : ',e)
       
        #suivre_formation
        val_suivre_formation=(current_code_formation,current_id_candidat,None,None,None,None)
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_suivre_formation, val_suivre_formation)
            mydb.commit()
        except Error as e :
            print('Error : ',e)
  
         #langues
        current_id_langues = global_id_generator
        val_langues=(current_id_langues,' '.join(cv['langues']),None)
        try :
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_langues, val_langues)
            mydb.commit()
            global_id_generator += 1
        except Error as e :
            print('Error : ',e)
            
        #parler_langues
        val_parler_langues=(current_id_langues,current_id_candidat,None,None)#must be extracted from the cv and checked in the database
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_parler_langues, val_parler_langues)
            mydb.commit()
        except Error as e :
            print('Error : ',e)
    
        #centre_interet
        current_id_centre_interet = global_id_generator
        val_centre_interet=(current_id_centre_interet,' '.join(cv['interet']))
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_centre_interet, val_centre_interet)
            mydb.commit()
            global_id_generator += 1
        except Error as e :
            print('Error : ',e)

        #aimer_interet
        val_aimer_interet=(current_id_centre_interet ,current_id_candidat, None)
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_aimer_interet, val_aimer_interet)
            mydb.commit()
 
       except Error as e :
           print('Error : ',e)
   
        #certificat
        current_id_certificat = global_id_generator
        val_certificat=(current_id_certificat,None,' '.join(cv['certificats']),None,None,None,current_id_candidat)
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_certificat, val_certificat)
            mydb.commit()
            global_id_generator += 1

        except Error as e :
            print('Error : ',e)
         
        #competences_operationnelles
        current_id_comp_op = global_id_generator
        val_competence_operationnelles = (current_id_comp_op,None,' '.join(cv['atouts']))
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_competence_operationnelles, val_competence_operationnelles)
            mydb.commit()
            global_id_generator += 1
    
        except Error as e :
            print('Error : ',e)
        
        #posseder_comp_op
        val_posseder_comp_op = (current_id_comp_op,current_id_candidat,None)
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_posseder_comp_op, val_posseder_comp_op)
            mydb.commit()
    
        except Error as e :
            print('Error : ',e)

       #competences_techniques
        current_id_comp_tech = global_id_generator
        val_competences_techniques = (current_id_comp_tech,' '.join(cv['competences']),None)
        try :
  
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_competence_technique, val_competences_techniques)
            mydb.commit()
            global_id_generator += 1
    
        except Error as e :
            print('Error : ',e)
 
       #posseder_comp_te
        val_posseder_comp_te = (current_id_comp_tech,current_id_candidat,None,None)
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_posseder_comp_te, val_posseder_comp_te)
            mydb.commit()
    
        except Error as e :
            print('Error : ',e)
  
       #reference
        current_id_reference = global_id_generator 
        val_reference=(current_id_reference,' '.join(cv['references']),None,None,None)
        try :
   
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_reference, val_reference)
            mydb.commit()
            global_id_generator += 1
   
       except Error as e :
           print('Error : ',e)
       
       #relation
        val_relation=(current_id_reference,current_id_candidat)
        try :
  
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_relation, val_relation)
            mydb.commit()
    
        except Error as e :
            print('Error : ',e)
    
        #projets
        current_id_projet = global_id_generator
        val_projets=(current_id_projet,' '.join(cv['projets']),None,None,None,None,None,None,None,current_id_candidat)
        try :
    
            my_cursor.execute(sql_select_database,())
            mydb.commit()
            my_cursor.execute(sql_projets, val_projets)
            mydb.commit()
            global_id_generator += 1
        except Error as e :
            print('Error : ',e)
    
    
    my_cursor.close()         

In [40]:
age_string="mqdkfl88j"
age_number = re.findall(r'\d+', age_string)
if(not age_number): 
    age_number=None
print(int(age_number[0]))


88


In [106]:
insert_into_mysql_db(candidates_list,'localhost','root','hell')

inserted 0
inserted 1
inserted 2
inserted 3
inserted 4
inserted 5
inserted 6
inserted 7
inserted 8
inserted 9
inserted 10
inserted 11
inserted 12
inserted 13
inserted 14
inserted 15
inserted 16
inserted 17
inserted 18
inserted 19
inserted 20
inserted 21
inserted 22
inserted 23
inserted 24
inserted 25
inserted 26
inserted 27
inserted 28
inserted 29
inserted 30
inserted 31
inserted 32
inserted 33
inserted 34
inserted 35
inserted 36
inserted 37
